# ML Model Training: Preferences → Cost Parameters

Train a regression model to translate user preferences into ODL cost parameters.

## Goal:
- **Input:** User preferences (parking, time, distance importance)
- **Output:** Cost parameters (costPerTravelHour, costPerKm, parking_multiplier)

## Data:
- **12,000 training samples** from Pareto Top-N selection
- Each preference matched to multiple cost/solution pairs
- Weighted by distance (closer = more important)

## Approach:
1. Load training data with sample weights
2. Train multiple regression models (with weighted samples)
3. Evaluate and compare
4. Save best model for deployment

## 1. Imports

In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import pickle
import os
from datetime import datetime

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Create output directory with timestamp
TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
OUTPUT_DIR = f'ml_training_results_{TIMESTAMP}'
FIGURES_DIR = os.path.join(OUTPUT_DIR, 'figures')
os.makedirs(FIGURES_DIR, exist_ok=True)

# Figure counter for auto-naming
figure_counter = {'count': 0}

def save_figure(name=None, dpi=300):
    """Automatically save the current matplotlib figure"""
    if name is None:
        figure_counter['count'] += 1
        name = f'figure_{figure_counter["count"]:02d}'
    
    filepath = os.path.join(FIGURES_DIR, f'{name}.png')
    plt.savefig(filepath, dpi=dpi, bbox_inches='tight')
    print(f"  💾 Saved: {filepath}")
    return filepath

print("✓ Imports successful")
print(f"✓ Output directory: {OUTPUT_DIR}")
print(f"✓ Figures will be saved to: {FIGURES_DIR}")

✓ Imports successful
✓ Output directory: ml_training_results_20251106_085226
✓ Figures will be saved to: ml_training_results_20251106_085226/figures


## 2. Configuration

In [ ]:
# Data file
TRAINING_DATA_FILE = 'training_data.json'

# Model save path
MODEL_SAVE_PATH = 'preference_to_cost_model.pkl'
SCALER_SAVE_PATH = 'preference_scaler.pkl'

# Train/test split
TEST_SIZE = 0.2
RANDOM_STATE = 42

print(f"Configuration:")
print(f"  Training data: {TRAINING_DATA_FILE}")
print(f"  Test size: {TEST_SIZE*100:.0f}%")
print(f"  Model output: {MODEL_SAVE_PATH}")

## 3. Load and Explore Data

In [ ]:
# Load training data
with open(TRAINING_DATA_FILE, 'r') as f:
    training_data = json.load(f)

print(f"Loaded {len(training_data)} training samples")

# Show first sample
print("\nSample structure:")
print(json.dumps(training_data[0], indent=2))

In [ ]:
# Convert to DataFrame for easier analysis
data = []
for sample in training_data:
    row = {
        # Inputs
        'parking_importance': sample['preferences']['parking_importance'],
        'time_importance': sample['preferences']['time_importance'],
        'distance_importance': sample['preferences']['distance_importance'],
        # Outputs
        'costPerTravelHour': sample['costs']['costPerTravelHour'],
        'costPerKm': sample['costs']['costPerKm'],
        'parking_multiplier': sample['costs']['parking_multiplier'],
        # Metadata
        'pool_id': sample['metadata']['pool_id'],
        'pareto_distance': sample['metadata']['pareto_distance'],
        'rank_for_preference': sample['metadata']['rank_for_preference'],
        # Sample weight
        'sample_weight': sample['sample_weight']
    }
    data.append(row)

df = pd.DataFrame(data)
print(f"\nDataFrame shape: {df.shape}")
df.head()

## 4. Data Analysis

In [ ]:
# Basic statistics
print("Dataset Statistics:\n")
print(df.describe())

# Check output diversity
unique_costs = df[['costPerTravelHour', 'costPerKm', 'parking_multiplier']].drop_duplicates()
print(f"\nUnique cost combinations: {len(unique_costs)}")
print(f"Average samples per unique output: {len(df) / len(unique_costs):.1f}")

# Check preference diversity
unique_prefs = df[['parking_importance', 'time_importance', 'distance_importance']].drop_duplicates()
print(f"Unique preference combinations: {len(unique_prefs)}")
print(f"Average samples per unique preference: {len(df) / len(unique_prefs):.1f}")

In [ ]:
# Analyze sample weights
print("Sample Weight Statistics:")
print(f"  Mean: {df['sample_weight'].mean():.3f}")
print(f"  Median: {df['sample_weight'].median():.3f}")
print(f"  Min: {df['sample_weight'].min():.3f}")
print(f"  Max: {df['sample_weight'].max():.3f}")

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.hist(df['sample_weight'], bins=50, edgecolor='black', alpha=0.7)
plt.title('Sample Weight Distribution')
plt.xlabel('Weight')
plt.ylabel('Count')

plt.subplot(1, 2, 2)
plt.hist(df['pareto_distance'], bins=50, edgecolor='black', alpha=0.7, color='orange')
plt.title('Pareto Distance Distribution')
plt.xlabel('Distance')
plt.ylabel('Count')

plt.tight_layout()
save_figure('sample_weights_and_distances')
plt.show()

print("\n✓ Sample weights give higher importance to closer matches")

In [ ]:
# Visualize input distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(df['parking_importance'], bins=30, edgecolor='black', alpha=0.7)
axes[0].set_title('Parking Importance Distribution')
axes[0].set_xlabel('Importance')

axes[1].hist(df['time_importance'], bins=30, edgecolor='black', alpha=0.7)
axes[1].set_title('Time Importance Distribution')
axes[1].set_xlabel('Importance')

axes[2].hist(df['distance_importance'], bins=30, edgecolor='black', alpha=0.7)
axes[2].set_title('Distance Importance Distribution')
axes[2].set_xlabel('Importance')

plt.tight_layout()
save_figure('input_distributions')
plt.show()

print("✓ Inputs are well-distributed (Dirichlet sampling)")

In [ ]:
# Visualize output distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(df['costPerTravelHour'], bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[0].set_title(f'Cost Per Travel Hour\n({len(df["costPerTravelHour"].unique())} unique values)')
axes[0].set_xlabel('Cost ($/hour)')

axes[1].hist(df['costPerKm'], bins=30, edgecolor='black', alpha=0.7, color='green')
axes[1].set_title(f'Cost Per Km\n({len(df["costPerKm"].unique())} unique values)')
axes[1].set_xlabel('Cost ($/km)')

axes[2].hist(df['parking_multiplier'], bins=30, edgecolor='black', alpha=0.7, color='red')
axes[2].set_title(f'Parking Multiplier\n({len(df["parking_multiplier"].unique())} unique values)')
axes[2].set_xlabel('Multiplier')

plt.tight_layout()
save_figure('output_distributions')
plt.show()

In [ ]:
# Analyze rank distribution (how many top-1, top-2, top-3 etc matches)
rank_counts = df['rank_for_preference'].value_counts().sort_index()
print("Distribution of Ranks (Top-N matches per preference):")
for rank, count in rank_counts.items():
    print(f"  Rank {rank}: {count} samples ({count/len(df)*100:.1f}%)")

plt.figure(figsize=(10, 5))
plt.bar(rank_counts.index, rank_counts.values, edgecolor='black', alpha=0.7)
plt.title('Distribution of Sample Ranks')
plt.xlabel('Rank (1 = closest match)')
plt.ylabel('Number of Samples')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
save_figure('rank_distribution')
plt.show()

## 5. Prepare Data for Training

In [ ]:
# Extract features (X) and targets (y)
X = df[['parking_importance', 'time_importance', 'distance_importance']].values
y = df[['costPerTravelHour', 'costPerKm', 'parking_multiplier']].values

# Extract sample weights
sample_weights = df['sample_weight'].values

print(f"Input shape (X): {X.shape}")
print(f"Output shape (y): {y.shape}")
print(f"Sample weights shape: {sample_weights.shape}")
print(f"\n✓ Ready for training with weighted samples")

In [ ]:
# Train/test split
X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    X, y, sample_weights, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"\nTest set is {TEST_SIZE*100:.0f}% of data")

In [ ]:
# For Neural Network: scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✓ Features scaled for Neural Network")

## 6. Train Multiple Models

In [ ]:
# Define models to train
models = {
    'Random Forest': RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        min_samples_split=5,
        random_state=RANDOM_STATE,
        n_jobs=-1
    ),
    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        random_state=RANDOM_STATE
    ),
    'Neural Network': MLPRegressor(
        hidden_layer_sizes=(64, 32, 16),
        activation='relu',
        max_iter=500,
        random_state=RANDOM_STATE,
        early_stopping=True
    )
}

print("Models to train:")
for name in models.keys():
    print(f"  - {name}")

In [ ]:
# Train models with sample weights
results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Wrap in MultiOutputRegressor
    multi_model = MultiOutputRegressor(model)
    
    # Use scaled data for Neural Network, regular data for others
    if name == 'Neural Network':
        multi_model.fit(X_train_scaled, y_train, sample_weight=weights_train)
        y_pred = multi_model.predict(X_test_scaled)
    else:
        multi_model.fit(X_train, y_train, sample_weight=weights_train)
        y_pred = multi_model.predict(X_test)
    
    # Calculate metrics for each output
    output_names = ['costPerTravelHour', 'costPerKm', 'parking_multiplier']
    metrics = {}
    
    for i, output_name in enumerate(output_names):
        mse = mean_squared_error(y_test[:, i], y_pred[:, i])
        mae = mean_absolute_error(y_test[:, i], y_pred[:, i])
        r2 = r2_score(y_test[:, i], y_pred[:, i])
        
        metrics[output_name] = {
            'MSE': mse,
            'RMSE': np.sqrt(mse),
            'MAE': mae,
            'R²': r2
        }
    
    # Overall R²
    overall_r2 = r2_score(y_test, y_pred, multioutput='uniform_average')
    
    results[name] = {
        'model': multi_model,
        'predictions': y_pred,
        'metrics': metrics,
        'overall_r2': overall_r2
    }
    
    print(f"  Overall R² score: {overall_r2:.4f}")

print("\n✓ All models trained with sample weights!")

## 7. Compare Models

In [ ]:
# Create comparison table
comparison_data = []

for model_name, result in results.items():
    for output_name, metrics in result['metrics'].items():
        comparison_data.append({
            'Model': model_name,
            'Output': output_name,
            'R²': metrics['R²'],
            'RMSE': metrics['RMSE'],
            'MAE': metrics['MAE']
        })

comparison_df = pd.DataFrame(comparison_data)
print("\nDetailed Model Comparison:\n")
print(comparison_df.to_string(index=False))

# Overall comparison
print("\n" + "="*60)
print("Overall R² Scores:")
for model_name, result in results.items():
    print(f"  {model_name:20s}: {result['overall_r2']:.4f}")
print("="*60)

In [ ]:
# Visualize R² scores
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: R² by model and output
pivot_df = comparison_df.pivot(index='Output', columns='Model', values='R²')
pivot_df.plot(kind='bar', ax=axes[0], edgecolor='black', alpha=0.8)
axes[0].set_title('R² Score by Model and Output', fontsize=12, fontweight='bold')
axes[0].set_ylabel('R² Score')
axes[0].set_xlabel('Output Parameter')
axes[0].legend(title='Model')
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha='right')

# Plot 2: Overall R²
overall_r2_data = [result['overall_r2'] for result in results.values()]
axes[1].bar(results.keys(), overall_r2_data, edgecolor='black', alpha=0.8)
axes[1].set_title('Overall R² Score by Model', fontsize=12, fontweight='bold')
axes[1].set_ylabel('R² Score')
axes[1].set_xlabel('Model')
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()
save_figure('model_comparison_r2')
plt.show()

In [ ]:
# Select best model based on overall R²
best_model_name = max(results.items(), key=lambda x: x[1]['overall_r2'])[0]
best_model = results[best_model_name]['model']
best_r2 = results[best_model_name]['overall_r2']

print(f"\n🏆 Best Model: {best_model_name}")
print(f"   Overall R² Score: {best_r2:.4f}")
print("\nPer-output performance:")
for output_name, metrics in results[best_model_name]['metrics'].items():
    print(f"  {output_name}:")
    print(f"    R² = {metrics['R²']:.4f}")
    print(f"    MAE = {metrics['MAE']:.4f}")

## 8. Prediction Analysis

In [ ]:
# Actual vs Predicted plots
y_pred_best = results[best_model_name]['predictions']
output_names = ['costPerTravelHour', 'costPerKm', 'parking_multiplier']

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, (ax, name) in enumerate(zip(axes, output_names)):
    ax.scatter(y_test[:, i], y_pred_best[:, i], alpha=0.3, s=10)
    
    # Perfect prediction line
    min_val = min(y_test[:, i].min(), y_pred_best[:, i].min())
    max_val = max(y_test[:, i].max(), y_pred_best[:, i].max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect prediction')
    
    ax.set_xlabel('Actual')
    ax.set_ylabel('Predicted')
    ax.set_title(f'{name}\n(R² = {results[best_model_name]["metrics"][name]["R²"]:.4f})')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.suptitle(f'{best_model_name} - Actual vs Predicted', fontsize=14, fontweight='bold')
plt.tight_layout()
save_figure('actual_vs_predicted')
plt.show()

print("✓ Points close to red line = good predictions")

In [ ]:
# Residual plots
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, (ax, name) in enumerate(zip(axes, output_names)):
    residuals = y_test[:, i] - y_pred_best[:, i]
    ax.scatter(y_pred_best[:, i], residuals, alpha=0.3, s=10)
    ax.axhline(y=0, color='r', linestyle='--', lw=2)
    ax.set_xlabel('Predicted Value')
    ax.set_ylabel('Residuals')
    ax.set_title(f'{name} Residuals')
    ax.grid(True, alpha=0.3)

plt.suptitle(f'{best_model_name} - Residual Analysis', fontsize=14, fontweight='bold')
plt.tight_layout()
save_figure('residual_analysis')
plt.show()

print("✓ If residuals are randomly scattered around 0, the model is unbiased")

## 9. Feature Importance (for tree-based models)

In [ ]:
# Feature importance for Random Forest or Gradient Boosting
if best_model_name in ['Random Forest', 'Gradient Boosting']:
    feature_names = ['parking_importance', 'time_importance', 'distance_importance']
    
    # Average feature importances across all outputs
    importances = np.mean([estimator.feature_importances_ 
                          for estimator in best_model.estimators_], axis=0)
    
    # Plot
    plt.figure(figsize=(8, 5))
    plt.bar(feature_names, importances, edgecolor='black', alpha=0.7)
    plt.title(f'Feature Importance - {best_model_name}')
    plt.ylabel('Importance')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    save_figure('feature_importance')
    plt.show()
    
    print("Feature Importances:")
    for name, imp in zip(feature_names, importances):
        print(f"  {name}: {imp:.4f}")
else:
    print(f"Feature importance not available for {best_model_name}")

## 10. Test on New Preferences

In [ ]:
# Test predictions on some example preferences
test_cases = [
    {'name': 'High parking priority', 
     'parking_importance': 0.8, 'time_importance': 0.1, 'distance_importance': 0.1},
    {'name': 'High time priority', 
     'parking_importance': 0.1, 'time_importance': 0.8, 'distance_importance': 0.1},
    {'name': 'High distance priority', 
     'parking_importance': 0.1, 'time_importance': 0.1, 'distance_importance': 0.8},
    {'name': 'Balanced priorities', 
     'parking_importance': 0.33, 'time_importance': 0.33, 'distance_importance': 0.34}
]

print("Test Predictions:\n")

for case in test_cases:
    X_new = np.array([[case['parking_importance'], 
                      case['time_importance'], 
                      case['distance_importance']]])
    
    # Scale if using Neural Network
    if best_model_name == 'Neural Network':
        X_new = scaler.transform(X_new)
    
    prediction = best_model.predict(X_new)[0]
    
    print(f"{case['name']}:")
    print(f"  Input: parking={case['parking_importance']:.2f}, "
          f"time={case['time_importance']:.2f}, "
          f"distance={case['distance_importance']:.2f}")
    print(f"  Predicted costs:")
    print(f"    costPerTravelHour = {prediction[0]:.2f}")
    print(f"    costPerKm = {prediction[1]:.4f}")
    print(f"    parking_multiplier = {prediction[2]:.2f}")
    print()

## 11. Save Best Model

In [ ]:
# Save the best model
model_path = os.path.join(OUTPUT_DIR, MODEL_SAVE_PATH)
with open(model_path, 'wb') as f:
    pickle.dump(best_model, f)

print(f"✓ Best model ({best_model_name}) saved to: {model_path}")

# Save scaler if using Neural Network
if best_model_name == 'Neural Network':
    scaler_path = os.path.join(OUTPUT_DIR, SCALER_SAVE_PATH)
    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)
    print(f"✓ Scaler saved to: {scaler_path}")

# Save metadata
metadata = {
    'model_type': best_model_name,
    'overall_r2': float(best_r2),
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'unique_outputs': int(len(unique_costs)),
    'unique_preferences': int(len(unique_prefs)),
    'used_sample_weights': True,
    'metrics': {name: {k: float(v) for k, v in metrics.items()} 
                for name, metrics in results[best_model_name]['metrics'].items()},
    'input_features': ['parking_importance', 'time_importance', 'distance_importance'],
    'output_features': ['costPerTravelHour', 'costPerKm', 'parking_multiplier'],
    'requires_scaling': best_model_name == 'Neural Network',
    'timestamp': TIMESTAMP
}

metadata_path = os.path.join(OUTPUT_DIR, 'model_metadata.json')
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✓ Metadata saved to: {metadata_path}")

## 12. Model Usage Example

In [ ]:
# Example: How to load and use the saved model
print("Example usage code:\n")
print("""
# Load model
import pickle
import numpy as np

with open('preference_to_cost_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Optional: Load scaler if using Neural Network
# with open('preference_scaler.pkl', 'rb') as f:
#     scaler = pickle.load(f)

# Get user preferences
preferences = {
    'parking_importance': 0.7,
    'time_importance': 0.2,
    'distance_importance': 0.1
}

# Prepare input
X = np.array([[
    preferences['parking_importance'],
    preferences['time_importance'],
    preferences['distance_importance']
]])

# Scale if needed
# X = scaler.transform(X)  # Only for Neural Network

# Predict costs
costs = model.predict(X)[0]

result = {
    'costPerTravelHour': costs[0],
    'costPerKm': costs[1],
    'parking_multiplier': costs[2]
}

print(result)
""")

## 13. Package Everything 📦

In [ ]:
import shutil

# Create a summary report
summary_report = f"""# ML Model Training Results

**Training Date:** {TIMESTAMP}
**Output Directory:** {OUTPUT_DIR}

---

## Dataset Summary

- **Total Training Samples:** {len(training_data):,}
- **Training Set:** {len(X_train):,} samples
- **Test Set:** {len(X_test):,} samples
- **Unique Cost Combinations:** {len(unique_costs):,}
- **Unique Preference Combinations:** {len(unique_prefs):,}
- **Used Sample Weights:** Yes (distance-based weighting)

---

## Model Performance

### Best Model: {best_model_name}

**Overall R² Score:** {best_r2:.4f}

### Per-Output Performance:

"""

for output_name, metrics in results[best_model_name]['metrics'].items():
    summary_report += f"""
#### {output_name}
- R² Score: {metrics['R²']:.4f}
- RMSE: {metrics['RMSE']:.4f}
- MAE: {metrics['MAE']:.4f}
"""

summary_report += f"""
---

## Model Comparison

| Model | Overall R² |
|-------|------------|
"""

for model_name, result in sorted(results.items(), key=lambda x: x[1]['overall_r2'], reverse=True):
    summary_report += f"| {model_name} | {result['overall_r2']:.4f} |\n"

summary_report += f"""
---

## Files Generated

### Model Files
- `{MODEL_SAVE_PATH}` - Trained model (pickle format)
"""

if best_model_name == 'Neural Network':
    summary_report += f"- `{SCALER_SAVE_PATH}` - Feature scaler (required for predictions)\n"

summary_report += f"""- `model_metadata.json` - Model metadata and metrics

### Figures ({figure_counter['count']} total)
"""

# List all saved figures
figure_files = sorted([f for f in os.listdir(FIGURES_DIR) if f.endswith('.png')])
for fig_file in figure_files:
    summary_report += f"- `figures/{fig_file}`\n"

summary_report += """
---

## Usage Instructions

```python
import pickle
import numpy as np

# Load the model
with open('preference_to_cost_model.pkl', 'rb') as f:
    model = pickle.load(f)
"""

if best_model_name == 'Neural Network':
    summary_report += """
# Load the scaler (Neural Network only)
with open('preference_scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
"""

summary_report += """
# Define user preferences
preferences = np.array([[
    0.7,  # parking_importance
    0.2,  # time_importance
    0.1   # distance_importance
]])
"""

if best_model_name == 'Neural Network':
    summary_report += """
# Scale inputs (Neural Network only)
preferences = scaler.transform(preferences)
"""

summary_report += """
# Predict costs
costs = model.predict(preferences)[0]

print(f"costPerTravelHour: {costs[0]:.2f}")
print(f"costPerKm: {costs[1]:.4f}")
print(f"parking_multiplier: {costs[2]:.2f}")
```

---

## Notes

- Model was trained with sample weights based on Pareto distance
- Closer preference-cost matches received higher importance during training
- All figures show model performance on the test set
- Ready for production deployment
"""

# Save summary report
summary_path = os.path.join(OUTPUT_DIR, 'TRAINING_SUMMARY.md')
with open(summary_path, 'w') as f:
    f.write(summary_report)

print(f"✅ Training summary saved to: {summary_path}")

# Create a zip file with everything
zip_filename = f'{OUTPUT_DIR}.zip'
shutil.make_archive(OUTPUT_DIR, 'zip', OUTPUT_DIR)

print(f"\n📦 All results packaged!")
print(f"\n{'='*60}")
print(f"OUTPUT DIRECTORY: {OUTPUT_DIR}")
print(f"{'='*60}")
print(f"\nContents:")
print(f"  📁 {MODEL_SAVE_PATH}")
if best_model_name == 'Neural Network':
    print(f"  📁 {SCALER_SAVE_PATH}")
print(f"  📁 model_metadata.json")
print(f"  📁 TRAINING_SUMMARY.md")
print(f"  📁 figures/ ({figure_counter['count']} images)")
print(f"\n  📦 {zip_filename} (all files packaged)")
print(f"\n{'='*60}")
print(f"\n✅ Training complete! All results saved and packaged.")
print(f"\n💡 Check {summary_path} for detailed results!")

## Summary

✅ Trained on 12,000 weighted samples from Pareto Top-N selection
✅ Sample weights prioritize closer preference-cost matches
✅ Evaluated Random Forest, Gradient Boosting, and Neural Network
✅ Saved best model for deployment
✅ Model translates preferences → cost parameters
✅ **All figures and model automatically saved to timestamped directory**
✅ **Complete package available as zip file**

### Key Findings:
- Training used weighted samples (closer matches = higher importance)
- Model learned from diverse preference-cost mappings
- Ready for production deployment
- All analysis figures saved for documentation

### Next Steps:
1. Integrate model into your application
2. Get user preferences (parking, time, distance importance)
3. Predict cost parameters using model
4. Send costs to ODL API for route optimization
5. Return optimized routes to user!